In [1]:
import pandas as pd
import numpy as np

In [2]:
df_accu = pd.read_excel('accuracy.xlsx')
drop = ['global_id', 'Line_en',
       'Year_en', 'Quarter_en', 'Precision_en']
df_accu.drop(labels=drop, axis=1, inplace = True)
df_accu.drop(labels = 0, axis=0, inplace = True)
df_accu = df_accu.reset_index(drop=True)

In [3]:
df_accu['Precision'] = df_accu['Precision'].astype(float)
df_accu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Line       144 non-null    object 
 1   Year       144 non-null    object 
 2   Quarter    144 non-null    object 
 3   Precision  144 non-null    float64
dtypes: float64(1), object(3)
memory usage: 4.6+ KB


In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_accu['Precision_normalized'] = scaler.fit_transform(df_accu[['Precision']])
df_accu

,Line,Year,Quarter,Precision,Precision_normalized
0,Сокольническая линия,2021,I квартал,99.99,0.947368
1,Замоскворецкая линия,2021,I квартал,99.99,0.947368
2,Арбатско-Покровская линия,2021,I квартал,99.99,0.947368
3,Филёвская линия,2021,I квартал,99.99,0.947368
4,Кольцевая линия,2021,I квартал,99.99,0.947368
...,...,...,...,...,...
139,Солнцевская линия,2023,I квартал,99.99,0.947368
140,Некрасовская линия,2023,I квартал,100.00,1.000000
141,Большая кольцевая линия,2023,I квартал,99.98,0.894737
142,Московская монорельсовая транспортная система,2023,I квартал,100.00,1.000000


In [5]:
df_traffic = pd.read_excel('passenger_traffic.xlsx')


/Users/ulanaderkac/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
dropp = ['global_id', 'NameOfStation_en', 'Line_en',
       'Year_en', 'Quarter_en', 'IncomingPassengers_en',
       'OutgoingPassengers_en']
df_traffic.drop(labels=dropp, axis=1, inplace = True)
df_traffic.drop(labels = 0, axis=0, inplace = True)
df_traffic = df_traffic.reset_index(drop=True)

In [7]:
df_traffic[['IncomingPassengers', 'OutgoingPassengers']] = df_traffic[['IncomingPassengers', 'OutgoingPassengers']].astype(int)

In [8]:
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520 entries, 0 to 2519
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NameOfStation       2520 non-null   object
 1   Line                2520 non-null   object
 2   Year                2520 non-null   object
 3   Quarter             2520 non-null   object
 4   IncomingPassengers  2520 non-null   int64 
 5   OutgoingPassengers  2520 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 118.2+ KB


In [23]:
df_traffic

,NameOfStation,Line,Year,Quarter,IncomingPassengers,OutgoingPassengers
0,Митино,Арбатско-Покровская линия,2021,I квартал,1913498,1829031
1,Волоколамская,Арбатско-Покровская линия,2021,I квартал,1236714,1222309
2,Строгино,Арбатско-Покровская линия,2021,I квартал,1938816,1903731
3,Крылатское,Арбатско-Покровская линия,2021,I квартал,1849616,1818208
4,Площадь Революции,Арбатско-Покровская линия,2021,I квартал,2324687,2319343
...,...,...,...,...,...,...
2515,Сокольники,Сокольническая линия,2023,I квартал,1892829,1916655
2516,Проспект Вернадского,Сокольническая линия,2023,I квартал,2129756,2141124
2517,Китай-город,Таганско-Краснопресненская линия,2023,I квартал,0,0
2518,Кунцевская,Филёвская линия,2023,I квартал,518577,502962


In [22]:
df_zero = df_traffic[df_traffic['IncomingPassengers'] == 0 ]
df_zero

,NameOfStation,Line,Year,Quarter,IncomingPassengers,OutgoingPassengers
22,Новослободская,Кольцевая линия,2021,I квартал,0,0
28,Рижская,Калужско-Рижская линия,2021,I квартал,0,0
133,Мнёвники,Большая кольцевая линия,2021,I квартал,0,0
150,Народное Ополчение,Большая кольцевая линия,2021,I квартал,0,0
309,Новослободская,Кольцевая линия,2021,II квартал,0,0
...,...,...,...,...,...,...
2393,Библиотека имени Ленина,Сокольническая линия,2023,I квартал,0,0
2494,Третьяковская,Калужско-Рижская линия,2023,I квартал,0,0
2514,Петровско-Разумовская,Серпуховско-Тимирязевская линия,2023,I квартал,0,0
2517,Китай-город,Таганско-Краснопресненская линия,2023,I квартал,0,0


In [21]:
df_zero['NameOfStation'].value_counts()


Петровско-Разумовская      7
Третьяковская              7
Китай-город                7
Киевская                   7
Тверская                   6
Библиотека имени Ленина    6
Театральная                6
Улица Старокачаловская     5
Рижская                    5
Новослободская             4
Каширская                  2
Смоленская                 2
Народное Ополчение         1
Мнёвники                   1
Технопарк                  1
Коломенская                1
Кантемировская             1
Царицыно                   1
Name: NameOfStation, dtype: int64

In [28]:
df_info = pd.read_excel('info.xlsx')


/Users/ulanaderkac/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
df_info.columns

Index(['Line', 'NumberOfStations', 'MetroLineLength', 'NumberOfCarriages',
       'global_id', 'Line_en', 'NumberOfStations_en', 'MetroLineLength_en',
       'NumberOfCarriages_en'],
      dtype='object')

In [11]:
droppp = ['global_id', 'Line_en', 'NumberOfStations_en', 'MetroLineLength_en',
       'NumberOfCarriages_en']
df_info.drop(labels=droppp, axis=1, inplace = True)
df_info.drop(labels = 0, axis=0, inplace = True)
df_info = df_info.reset_index(drop=True)

In [26]:
df_info

,Line,NumberOfStations,MetroLineLength,NumberOfCarriages
0,Сокольническая линия,26,41.34,509
1,Замоскворецкая линия,24,42.92,746
2,Арбатско-Покровская линия,22,44.98,479
3,Филёвская линия,13,14.53,162
4,Кольцевая линия,12,19.17,280
5,Калининская линия,8,16.22,248
6,Калужско-Рижская линия,24,37.83,671
7,Таганско-Краснопресненская линия,23,42,672
8,Серпуховско-Тимирязевская линия,25,41.31,696
9,Люблинско-Дмитровская линия,23,37.56,708


In [27]:
df_info[['NumberOfStations', 'NumberOfCarriages']] = df_info[['NumberOfStations', 'NumberOfCarriages']].astype(int)
df_info['MetroLineLength'] = df_info['MetroLineLength'].astype(float)
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Line               16 non-null     object 
 1   NumberOfStations   16 non-null     int64  
 2   MetroLineLength    16 non-null     float64
 3   NumberOfCarriages  16 non-null     int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 640.0+ bytes
